# Implied Volatility & the BSM Model

- The Black-Scholes-Merton model gives:

# $C\left ( S_{t}, K, t, r,\sigma  \right ) = S_{t}\cdot {\mathbf{N}}(d_{1})-e^{-r(T-t)}\cdot K\cdot \mathbf{N}(d_{2})$

where

# $\mathbf{N}(d) = \frac{1}{\sqrt{2\pi }}\int_{-\infty}^{d}e^{-\frac{1}{2}x^{2}}dx$

- $\mathbf{N}$ represents the cumulative normal dist'n function

# $d_{1} = \frac{log\left ( \frac{S_{t}}{K} \right ) + \left ( r+\frac{\sigma^{2}}{2} \right )\left ( T-t \right )}{\sigma \sqrt{T-t} }$

# $d_{2} = \frac{log\left ( \frac{S_{t}}{K} \right ) + \left ( r-\frac{\sigma^{2}}{2} \right )\left ( T-t \right )}{\sigma \sqrt{T-t} }$

- $S_{t}$ represents the price of stock $S$ at time $t$
- $\sigma$ represents the volatility of the price of $S$
- $K$ represents the strike price of the option on stock $S$
- $T$ represents the maturity of the option
- $r$ represents the constant riskless short rate i.e. RFR

- If we're given a quote $C^{*}$ for a European call option with price $S^{t}$, strike $K$, maturity $T$ with market RFR $r$, we can reverse engineer the value of $\sigma$ i.e. solve for the Implied Volatility

- There is no closed-form solution i.e. we can't isolate and solve for $\sigma$
    - must use a numerical solution

- The Newton scheme to estimate the correct solution iterates over the first derivative until a certain level of accuracy is achieved

## Newton scheme

# $\sigma^{implied}_{n+1} = \sigma^{implied}_{n} - \frac{C(\sigma^{implied}_{n})-C^{*}}{\partial C(\sigma^{implied}_{n})/\partial\sigma^{implied}_{n}}$

- The **Vega** of the option gives the partial derivative of $C$ wrt $\sigma$:

# $\frac{\partial C}{\partial \sigma} = S_{t} \cdot  \mathbf{N}(d_{1}) \cdot \sqrt{T-t} $

In [1]:
from math import log, sqrt, exp
from scipy import stats

In [2]:
def bsm_call_value(S0, K, T, r, sigma):
    #this function returns the value of C based on the inputs
    
    d1 = (log(float(S0)/K) + (r + 0.5*sigma**2)*T)/(sigma*sqrt(T))
    d2 = (log(float(S0)/K) + (r - 0.5*sigma**2)*T)/(sigma*sqrt(T))
    
    norm1 = stats.norm.cdf(d1,0,1)
    norm2 = stats.norm.cdf(d2,0,1)
    #stats.norm.cdf(a,0,1) returns the cdf for a normal dist'n with mean 0 and SD 1
    
    value = S0*norm1 - exp(-r*T)*K*norm2
    
    return value

In [5]:
def bsm_vega(S0, K, T, r, sigma):
    d1 = (log(float(S0)/K) + (r + 0.5*sigma**2)*T)/(sigma*sqrt(T))
    norm1 = stats.norm.cdf(d1,0,1)
    
    vega = S0 * norm1 * sqrt(T)
    
    return vega

In [18]:
def bsm_imp_vol(S0, K, T, r, C0, sigma_estimate, iterations):
    
    for i in range(iterations):
        print sigma_estimate
        numerator = bsm_call_value(S0, K, T, r, sigma_estimate) - C0
        denominator = bsm_vega(S0, K, T, r, sigma_estimate)
        
        sigma_estimate -= float(numerator)/denominator
        
    return sigma_estimate

**Example**

- Walgreens Boots Alliance, Inc. (WBA)
    - $C^{*}$ = $3.23
    
    - $S_{0}$ = $83.11
    
    - $K$ = 80
    
    - $r$ = 0.25%
    
    - $T$ = 1 day

### We need to guess an initial value for implied volatility
    - If we guess 0.3, see what value it spits out for C0

In [16]:
bsm_call_value(83.11,80,1/float(365),0.0025, 0.3)

3.1137364434604962

- 0.3 is a fine estimate
    - run bsm_imp_vol

In [19]:
bsm_imp_vol(83.11, 80, 1/float(365),0.0025,3.23, 0.3, 100)

0.3
0.326925515318
0.353274958887
0.378738671806
0.402985153561
0.425701556964
0.446630129928
0.465593390884
0.482504742148
0.497365293475
0.510250534978
0.52129159456
0.53065539114
0.53852673188
0.545093972301
0.550538688261
0.555029051995
0.558716231341
0.561733033684
0.564194078696
0.566196917304
0.567823662165
0.569142827628
0.570211183668
0.571075507194
0.571774169263
0.572338533084
0.572794160314
0.5731618362
0.573458430771
0.573697615929
0.573890458453
0.574045907753
0.574171195367
0.574272161057
0.574353518302
0.574419069975
0.574471883267
0.574514431357
0.574548708029
0.574576320327
0.574598563391
0.574616480879
0.574630913725
0.574642539466
0.574651903961
0.574659446959
0.574665522716
0.574670416606
0.57467435851
0.574677533599
0.574680091035
0.574682150964
0.574683810165
0.574685146591
0.574686223032
0.574687090065
0.574687788426
0.574688350928
0.574688804001
0.574689168933
0.574689462871
0.574689699626
0.574689890323
0.574690043921
0.574690167638
0.574690267287
0.5746903475

0.57469067959893194

- Testing the value of 0.57469067959893194 as the implied volatility

In [20]:
bsm_call_value(83.11,80,1/float(365),0.0025, 0.57469067959893194)

3.2299999997991478

- VERY CLOSE!